In [72]:
import pandas as pd
import numpy as np
import sklearn
import xgboost

In [73]:
raw_data = pd.read_csv("credit_risk_dataset.csv")
raw_data

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,N,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,N,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,N,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,N,26


In [74]:
raw_data["person_emp_length"].fillna(raw_data["person_emp_length"].median(), inplace=True)
raw_data["loan_int_rate"].fillna(raw_data["loan_int_rate"].median(), inplace=True)

In [75]:
raw_data.duplicated().sum()

165

In [76]:
raw_data.drop_duplicates(inplace=True)
raw_data.shape

(32416, 12)

In [77]:
raw_data = raw_data.rename(columns={"cb_person_default_on_file":"default_history",
                                    "cb_person_cred_hist_length": "cr_history_length"})

In [78]:

raw_data = raw_data[raw_data["person_age"] <= 100]

In [79]:
raw_data

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,default_history,cr_history_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,N,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,N,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,N,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,N,26


In [80]:
raw_data["default_history"] = raw_data["default_history"].map(dict(Y=1, N=0))

In [81]:
raw_data

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,default_history,cr_history_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,1,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,0,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,0,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,0,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,0,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,0,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,0,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,0,26


In [82]:
numerical_features = raw_data.select_dtypes(exclude='object')
categorical_features = raw_data.select_dtypes(include='object')

In [88]:
categorical_features

,person_home_ownership,loan_intent
0,RENT,PERSONAL
1,OWN,EDUCATION
2,MORTGAGE,MEDICAL
3,RENT,MEDICAL
4,RENT,MEDICAL
...,...,...
32576,MORTGAGE,PERSONAL
32577,MORTGAGE,PERSONAL
32578,RENT,HOMEIMPROVEMENT
32579,MORTGAGE,PERSONAL


In [87]:
categorical_features = categorical_features.drop(["loan_grade"], axis=1)

KeyError: "['loan_grade'] not found in axis"

In [89]:
one_encode_features = pd.get_dummies(categorical_features)

In [90]:
one_encode_features

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,0,0,0,1,0,0,0,0,1,0
1,0,0,1,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,1,0,0
4,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
32576,1,0,0,0,0,0,0,0,1,0
32577,1,0,0,0,0,0,0,0,1,0
32578,0,0,0,1,0,0,1,0,0,0
32579,1,0,0,0,0,0,0,0,1,0


In [98]:
features = pd.concat([numerical_features, one_encode_features], axis=1)

In [100]:
features

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,default_history,cr_history_length,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,22,59000,123.0,35000,16.02,1,0.59,1,3,0,0,0,1,0,0,0,0,1,0
1,21,9600,5.0,1000,11.14,0,0.10,0,2,0,0,1,0,0,1,0,0,0,0
2,25,9600,1.0,5500,12.87,1,0.57,0,3,1,0,0,0,0,0,0,1,0,0
3,23,65500,4.0,35000,15.23,1,0.53,0,2,0,0,0,1,0,0,0,1,0,0
4,24,54400,8.0,35000,14.27,1,0.55,1,4,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,1.0,5800,13.16,0,0.11,0,30,1,0,0,0,0,0,0,0,1,0
32577,54,120000,4.0,17625,7.49,0,0.15,0,19,1,0,0,0,0,0,0,0,1,0
32578,65,76000,3.0,35000,10.99,1,0.46,0,28,0,0,0,1,0,0,1,0,0,0
32579,56,150000,5.0,15000,11.48,0,0.10,0,26,1,0,0,0,0,0,0,0,1,0


In [101]:
features = features.drop([ "loan_status"], axis=1)

In [103]:
features

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,default_history,cr_history_length,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE
0,22,59000,123.0,35000,16.02,0.59,1,3,0,0,0,1,0,0,0,0,1,0
1,21,9600,5.0,1000,11.14,0.10,0,2,0,0,1,0,0,1,0,0,0,0
2,25,9600,1.0,5500,12.87,0.57,0,3,1,0,0,0,0,0,0,1,0,0
3,23,65500,4.0,35000,15.23,0.53,0,2,0,0,0,1,0,0,0,1,0,0
4,24,54400,8.0,35000,14.27,0.55,1,4,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,1.0,5800,13.16,0.11,0,30,1,0,0,0,0,0,0,0,1,0
32577,54,120000,4.0,17625,7.49,0.15,0,19,1,0,0,0,0,0,0,0,1,0
32578,65,76000,3.0,35000,10.99,0.46,0,28,0,0,0,1,0,0,1,0,0,0
32579,56,150000,5.0,15000,11.48,0.10,0,26,1,0,0,0,0,0,0,0,1,0


In [104]:
labels

,loan_status
0,1
1,0
2,1
3,1
4,1
...,...
32576,0
32577,0
32578,1
32579,0


In [105]:
# Split and create train/tests set
X = features
y = labels

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=99)

In [106]:
#model
import xgboost as xgb
xgb_clf = xgb.XGBClassifier()

# fit model
xgb_clf.fit(X_train, y_train)

# predict
xgb_preds = xgb_clf.predict(X_test)

/Users/arhumzafar/miniforge3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/arhumzafar/miniforge3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/arhumzafar/miniforge3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[15:18:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [107]:
import pickle
pickle_out = open("classifier.pkl", mode="wb")
pickle.dump(xgb_clf, pickle_out)
pickle_out.close()